# RFM 
- RFM 분석이란 CRM(Customer Relationship Management)을 할 때 사용하는 기법 중 하나인 RFM 분석으로
    - Recency : 얼마나 최근에 행동하였나?
        - 고객의 최근성이라고 하며 특정 행동을 얼마나 최근에 했었는지를 의미한다. (얼마나 최근에 구매하였는가?)
        - 최근성 관점에서 최근에 특정 행동을 취한 고객이 나중에 행동을 취한 고객보다 더 가치 있게 되는 것이다.
    - Frequency : 얼마나 자주 행동하였는가?
        - 고객의 행동빈도라고 하며 특정 행동을 얼마나 자주 했는지를 의미한다. (얼마나 자주 방문하였는가?)
        - 행동빈도 관점에서 정해진 기간 동안 고객이 특정 행동을 자주 할수록 가치 있는 고객이라 판단.
    - Monetary : 얼마나 많은 금액을 지출했는가
    
    즉 사용자별로 얼마나 최근에, 얼마나 자주, 얼마나 많은 금액을 지출했는지에 따라 사용자들의 분포를 확인하거나 사용자 그룹(또는 등급)을 나누어 분류하는 분석 기법이다.

## Dataset EDA

In [1]:
# 기본 Library 
import numpy as np
import pandas as pd
import datetime as dict
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
df = pd.read_excel('data/Online Retail.xlsx')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# 기본적인 Columns 정보 확인하기
# Description, CustomerID에 결측값이 존재한다.
# 총 7개의 컬럼 
# Invoice : 주문 
# StockCode : 제품 번호
# Description : 제품 설명
# Quantity : 양
# InvoiceDate : 주문일자
# UnitPrice : 제품당 가격
# CustomerID : 회원ID
# Country : 지역 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# 결측치 확인하기
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
# CustomerID의 결측값 제거하기
df = df.dropna(subset=['CustomerID'])
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [7]:
# 중복확인하기
df.duplicated().sum()

5225

In [8]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [9]:
# Columns의 정보를 확인한다.
# 정보를 확인한 결과 UnitPrice의 최소 가격이 0인 경우는 존재하지 않으며
# Quantity의 -80995또한 0미만의 값이 나올 수 없다. 
df.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


In [10]:
# Quantity와 UnitPrice의 0이하인 값들을 제거한다. 
df = df[(df['Quantity'] > 0) & (df['UnitPrice']>0)]
df.describe()

,Quantity,UnitPrice,CustomerID
count,392692.000000,392692.000000,392692.000000
mean,13.119702,3.125914,15287.843865
std,180.492832,22.241836,1713.539549
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13955.000000
50%,6.000000,1.950000,15150.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,8142.750000,18287.000000


In [11]:
df.shape

(392692, 8)

## RFM 분석 

In [12]:
# 구매 총액을 위한 Column 생성
df['TotalSum'] = df['UnitPrice'] * df['Quantity']

# 구매이력의 최소값과 최대값 출력하기 
print('Min Invocie Date:', df.InvoiceDate.dt.date.min(), 'max Invoice Date:', df.InvoiceDate.dt.date.max())
df.head(3)

Min Invocie Date: 2010-12-01 max Invoice Date: 2011-12-09


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


In [15]:
# 기간을 설정하기 위해서 Max값에 1일을 더해준다. 
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [ ]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [34]:
# rfm 데이터를 위해서 가공한다. 
# 여러개의 함수를 여러 열에 적용하는 agg()함수를 사용하여 rfm dataframe을 만든다. 
# Recency로 사용될 InvocieDate는 얼마나 예전에 구매했는지를 나타낸다.
# InvoiceNo는 빈도를 측정하기 위해서 count를 활용해 구매 횟수를 측정한다.
# Monetary 는 얼마나 많은 금액을 지출했는지 파악하기 위해서 sum으로 측정한다. 
rfm = df.groupby(['CustomerID']).agg({'InvoiceDate': lambda x : (snapshot_date - x.max()).days,
                                      'InvoiceNo':'count','TotalSum': 'sum'})

rfm.rename(columns={'InvoiceDate':'Recency', 'InvoiceNo':'Frequency', 'TotalSum':'MonetaryValue'}, inplace=True)
rfm.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


## 고객지표

- RFM 분석에서는 적절한 가중치를 통하여 고객을 평가하는 지표로 활용할 수 있다.
    
    $고객지표 = a * Recency + b*Frequency+c*Monetary$
    
- a, b, c에 대한 가중치는 고객 분류, 회사에 따라 다르며 중요한 것은 **분산이 높아 고객이 잘 분리된 고객 지표가 필요하다.**

```
Kaggle Notebook에서는 고객지표에 대해서 가중치를 다음과 같이 주었다.

각 회사는 고객이 최신까지 구매하기를 원하기 때문에 최근에 활동한 Recency 고객을 원한다.

고객이 더 많은 돈을 지출하고 방문하기를 원하기 때문에 Frequency와 Monetary Value를 더 높게 평가한다. (Recency와는 다른 순서이다. )
```


In [35]:
# Recency, Frequency, Monetary를 4개의 Level로 나눈다. 
# Pandas에서 제공하는 qcut을 통해서 구간을 나눈다. 
# qcut은 cut과 다르게 동일한 갯수로 구간을 나누는 함수이다. 
r_labels =range(4,0,-1)
f_labels=range(1,5)
m_labels=range(1,5)

r_quartiles = pd.qcut(rfm['Recency'], q=4, labels = r_labels)
f_quartiles = pd.qcut(rfm['Frequency'],q=4, labels = f_labels)
m_quartiles = pd.qcut(rfm['MonetaryValue'],q=4,labels = m_labels)

# assign함수를 사용해서 새 열을 할당한다. 
rfm = rfm.assign(R = r_quartiles, F=f_quartiles, M=m_quartiles)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346.0,326,1,77183.60,1,1,4
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [37]:
# RFM Segment 
def add_rfm(x) : return str(x['R']) + str(x['F']) + str(x['M'])
rfm['RFM_Segment'] = rfm.apply(add_rfm,axis=1 )
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,1,77183.60,1,1,4,114,6
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


In [39]:
# 고객별로 고객지표 및 분류가 완성되었다. 
# Segement 별로 고객의 수를 확인한다. 
rfm.groupby(['RFM_Segment']).size().sort_values(ascending=False)

RFM_Segment
444    450
111    381
344    217
122    206
211    179
      ... 
124      7
314      7
414      6
142      3
441      3
Length: 61, dtype: int64

In [41]:
# 가장 레벨이 높은 444의 고객을 확인해보기 
rfm[rfm['RFM_Segment']=='444']

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12347.0,2,182,4310.00,4,4,4,444,12
12362.0,3,266,5226.23,4,4,4,444,12
12388.0,16,100,2780.66,4,4,4,444,12
12417.0,3,192,3649.10,4,4,4,444,12
12423.0,1,125,1859.31,4,4,4,444,12
...,...,...,...,...,...,...,...,...
18229.0,12,164,7276.90,4,4,4,444,12
18241.0,10,104,2073.09,4,4,4,444,12
18245.0,7,175,2567.06,4,4,4,444,12


In [42]:
# 고객 지표별로 세부 사항 확인하기
# 고객지표가 높으면 높을수록 다른 낮은 지표의 고객군에 비해서 Recency는 확연히 작고 Frequency와 MonetaryValue는 높은것을 확인할 수 있다.
rfm.groupby('RFM_Score').agg({'Recency':'mean', 'Frequency':'mean', 'MonetaryValue':['mean', 'count']}).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           260.7       8.2         157.4   381
4           177.2      13.6         240.0   388
5           152.9      21.2         366.6   518
6            95.9      27.9         820.8   457
7            79.6      38.0         758.1   463
8            64.1      56.0         987.3   454
9            45.9      78.7        1795.1   414
10           32.4     110.5        2056.4   426
11           21.3     186.9        4062.0   387
12            7.2     367.8        9285.9   450